In [54]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])


def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std


X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [55]:
def calc_logloss(y, y_pred, eps=10**(-8)):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return - np.mean(y * np.log(y_pred) + (1. - y) * np.log(1. - y_pred))

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [56]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [57]:
def eval_model(X, y, iterations, eta=10**(-4)):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        dQ = 1. / n * X.T @ (y_pred - y)
        W -= eta * dQ
    return W, calc_logloss(y, y_pred)

In [58]:
iterations = np.logspace(1, 2, 4, dtype=np.int)
etas = np.linspace(10**(-2), 5, 10)
best_error = np.inf
best_params = {}
for iteration in tqdm(iterations):
    for eta in etas:
        W, error = eval_model(X_st, y, iteration, eta)
        if error < best_error:
            best_error = error
            best_params = {
                'iterations': iteration,
                'eta': eta
            }
print(f'Ошибка {best_error} при параметрах {best_params}')

C:\TMP/ipykernel_14592/1468952157.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  iterations = np.logspace(1, 2, 4, dtype=np.int)
100%|██████████| 4/4 [00:00<00:00, 34.50it/s]

Ошибка 0.20302333927708918 при параметрах {'iterations': 100, 'eta': 5.0}


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [59]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(X, W))

In [60]:
calc_pred_proba(W, X_st)

array([3.58043233e-01, 1.13341370e-01, 9.99605449e-01, 4.39572170e-04,
       7.46305694e-01, 2.33000018e-01, 9.99914868e-01, 8.92559045e-02,
       4.46315218e-01, 9.97517198e-01])

4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.
Могла ли модель переобучиться? Почему?

In [61]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [62]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

In [63]:
def accuracy(y, y_pred):
    return np.mean(y == y_pred)

In [64]:
accuracy(y, pred)

0.9

In [65]:
def confusion_matrix(y, y_pred):
    c_matrix = np.zeros((2, 2))
    for i in range(len(y)):
        if y[i] == y_pred[i] == 1:
            c_matrix[0][0] += 1
        elif y[i] == y_pred[i] == 0:
            c_matrix[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1:
            c_matrix[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0:
            c_matrix += 1
    return c_matrix

In [66]:
c_matrix = confusion_matrix(y, pred)
c_matrix

array([[4., 0.],
       [1., 5.]])

In [67]:
def precision(y, y_pred):
    c_matrix = confusion_matrix(y, y_pred)
    return c_matrix[0][0] / (c_matrix[0][0] + c_matrix[0][1])

In [68]:
precision(y, pred)

1.0

In [69]:
def recall(y, y_pred):
    c_matrix = confusion_matrix(y, y_pred)
    return c_matrix[0][0] / (c_matrix[0][0] + c_matrix[1][0])

In [70]:
recall(y, pred)

0.8

In [71]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    return 2. * pr * rec / (pr + rec)

In [72]:
f_score(y, pred)

0.888888888888889

На такой маленькой выборке трудно сделать вывод о переобучении. Судя по значениям метрик, модель практически безошибочно предсказывает $y$ для тренировочных данных. Разделим данные на тест и трейн:

In [75]:
split_ind = 5
X_train, X_test = X[:split_ind], X[split_ind:]
y_train, y_test = y[:split_ind], y[split_ind:]
X_train_st = X_train.copy().astype(np.float64)
X_train_st[:, 1:4] = standard_scale(X_train_st[:, 1:4])
X_test_st = X_test.copy().astype(np.float64)
X_test_st[:, 1:4] = standard_scale(X_test_st[:, 1:4])

In [76]:
W_1, error_1 = eval_model(X_train, y_train, 100, 5)
pred_train = calc_pred_proba(W_1, X_train_st)
pred_test = calc_pred_proba(W_1, X_test_st)
print('Train:')
print(f'Accuracy: {accuracy(y_train, pred_train)}, precision: {precision(y_train, pred_train)}, recall: {recall(y_train, pred_train)}, fscore: {f_score(y_train, pred_train)}')
print(f'Confusion matrix:\n{confusion_matrix(y_train, pred_train)}')
print('Test:')
print(f'Accuracy: {accuracy(y_test, pred_test)}, precision: {precision(y_test, pred_test)}, recall: {recall(y_test, pred_test)}, fscore: {f_score(y_test, pred_test)}')
print(f'Confusion matrix:\n{confusion_matrix(y_test, pred_test)}')

Train:
Accuracy: 0.2, precision: 0.5714285714285714, recall: 0.5, fscore: 0.5333333333333333
Confusion matrix:
[[4. 3.]
 [4. 3.]]
Test:
Accuracy: 0.6, precision: 0.75, recall: 0.6, fscore: 0.6666666666666665
Confusion matrix:
[[3. 1.]
 [2. 2.]]


C:\TMP/ipykernel_14592/3196251242.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Предсказания на тесте не слишком сильно отличаются от трейна. Скорее всего, пока модель не переобучена, но на выборке из суммарно 10 значений такие выводы делать сложно.